In [1]:
import statsmodels.stats.multitest as smm

In [14]:
import numpy as np
import pandas as pd
import scipy
from scipy import stats
from statsmodels.stats.descriptivestats import sign_test
from statsmodels.stats.weightstats import *

In [3]:
df = pd.read_csv("gene_high_throughput_sequencing.csv", header = 0)


Часть 1: применение t-критерия Стьюдента

В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

    для групп normal (control) и early neoplasia (treatment)
    для групп early neoplasia (control) и cancer (treatment)

В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости. 

In [4]:
df.drop('Patient_id',axis =1, inplace = True)

In [5]:
N = df[df.Diagnosis == 'normal']

In [6]:
N.drop('Diagnosis',axis =1, inplace = True)

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [7]:
E = df[df.Diagnosis == 'early neoplasia']

In [8]:
E.drop('Diagnosis',axis =1, inplace = True)

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
C = df[df.Diagnosis == 'cancer']

In [10]:
C.drop('Diagnosis',axis =1, inplace = True)

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
len(N.columns)

15748

Группы normal(N) - и early neoplasia(E)

In [16]:
NEp = []
c1 = 0 # счетчик
for i in range(len(N.columns)):
    _ ,p = scipy.stats.ttest_ind(N[N.columns[i]], E[E.columns[i]], equal_var = False)
    NEp.append([E.columns[i],p])
    if p < 0.05:
        c1 += 1

In [17]:
c1

1575

Группы early neoplasia(E) и cancer(C)

In [18]:
ECp = []
c2 = 0 # счетчик
for i in range(len(N.columns)):
    _ ,p = scipy.stats.ttest_ind(E[E.columns[i]], C[C.columns[i]], equal_var = False)
    ECp.append([E.columns[i],p])
    if p < 0.05:
        c2 += 1

In [19]:
c2

3490

Часть 2: поправка методом Холма
В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.5.

In [30]:
NEp2 = []
ind2 = []
F = [] # метрика fold change
c3 = 0 # счетчик
for i in range(len(N.columns)):
    _ ,p = scipy.stats.ttest_ind(N[N.columns[i]], E[E.columns[i]], equal_var = False)
    NEp2.append(p)
    if p < 0.025:
        c3 += 1
        ind2.append(i)
    if np.mean(N[N.columns[i]]) < np.mean(E[E.columns[i]]):
        F.append(np.mean(E[E.columns[i]]) / np.mean(N[N.columns[i]]))
    else:    
        F.append(np.mean(N[N.columns[i]])/np.mean(E[E.columns[i]]))

In [21]:
c3

992

In [22]:
reject1, p_corrected1, a1, a2 = smm.multipletests(NEp2, 
                                            alpha = 0.025, 
                                            method = 'holm') 

In [23]:
unique, counts = np.unique(reject1, return_counts=True)

In [24]:
print np.asarray((unique, counts)).T

[[    0 15746]
 [    1     2]]


In [25]:
c4 = 0
for i in range(len(p_corrected1)):
    if p_corrected1[i] < 0.025 and F[i] > 1.5:
        c4 += 1
    

In [26]:
c4

2

In [34]:
ECp2 = []
#ind3 = []
F1 = []
#c3 = 0 # счетчик
for i in range(len(N.columns)):
    _ ,p = scipy.stats.ttest_ind(E[E.columns[i]], C[C.columns[i]], equal_var = False)
    ECp2.append(p)
    if np.mean(E[E.columns[i]]) < np.mean(C[C.columns[i]]):
        F1.append(np.mean(C[C.columns[i]]) / np.mean(E[E.columns[i]]))
    else:    
        F1.append(np.mean(E[E.columns[i]])/np.mean(C[C.columns[i]]))
    

In [29]:
len(ECp2)

15748

In [35]:
reject2, p_corrected2, a12, a22 = smm.multipletests(ECp2, 
                                            alpha = 0.025, 
                                            method = 'holm') 

In [36]:
c5 = 0
for i in range(len(p_corrected2)):
    if p_corrected2[i] < 0.025 and F1[i] > 1.5:
        c5 += 1

In [37]:
c5

77

Часть 3: поправка методом Бенджамини-Хохберга

Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.

Обратите внимание, что методы коррекции, которые контролируют FDR, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от H0, когда они есть, и будут чаще отклонять H0, когда отличий нет).

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых abs(fold change) > 1.5. 

In [38]:
reject3, p_corrected3, a3, a4 = smm.multipletests(NEp2, 
                                            alpha = 0.025, 
                                            method = 'fdr_bh') 

In [39]:
c6 = 0
for i in range(len(p_corrected3)):
    if p_corrected3[i] < 0.025 and F[i] > 1.5:
        c6 += 1
c6

4

In [41]:
reject4, p_corrected4, a5, a6 = smm.multipletests(ECp2,
                                            alpha = 0.025, 
                                            method = 'fdr_bh') 

In [42]:
c7 = 0
for i in range(len(p_corrected4)):
    if p_corrected4[i] < 0.025 and F1[i] > 1.5:
        c7 += 1
c7

524